In [21]:
def lcs(X):
    n = len(X)

    # dp[i][j] = LCS length for X[:i] and Y[:j]
    dp = [[0] * (n + 1) for _ in range(n + 1)]

    # Fill the dp table (bottom-up)
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if X[i - 1] < X[j - 1]:
                dp[i][j] = dp[i - 1][i] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

    # Now reconstruct the LCS string by backtracking
    i, j = n-1, n
    lcs_chars = []

    while i > 0 and j > 0:
        # Characters match → this char is part of LCS
        if X[i - 1] < X[j - 1]:
            lcs_chars.append(X[j - 1])
            j=i
            i -= 1
           
        # Move to the larger of the two neighbors
        elif dp[i - 1][j] >= dp[i][j - 1]:
            i -= 1
        else:
            j -= 1
    lcs_chars.append(X[min(j,i)])
    # Reverse because we collected from the end
    lcs_string = list(reversed(lcs_chars))

    print("LCS length =", dp[n][n])
    print("LCS =", lcs_string)

    return dp[n][n], lcs_string

In [22]:
Y = [2,5,3,4,7,4,9,10,1]

lcs(Y)


LCS length = 6
LCS = [2, 3, 4, 7, 9, 10]


(6, [2, 3, 4, 7, 9, 10])

In [ ]:
def rod_cut_top_down(p, n):
    memo = [-1] * (n + 1)
    solution = [0] * (n + 1)

    def aux(length):
        if length == 0:
            return 0
        if memo[length] >= 0:
            return memo[length]
        best = -float('inf')
        for i in range(1, length + 1):
            value = p[i] + aux(length - i)
            if value > best:
                best = value
                solution[length] = i
        memo[length] = best
        return best

    best_value = aux(n)

    # reconstruct strategy
    cuts = []
    length = n
    while length > 0:
        cuts.append(solution[length])
        length -= solution[length]

    print("Top-down best revenue =", best_value)
    print("Cutting strategy =", cuts)
    return best_value, cuts
n=9
#np.array([1,2,3,4,5,6,7,8,9,10],dtype=int)
p=[0,1,5,8,9,10,17,17,20,24,30]
print(rod_cut_top_down(p, n))

In [1]:
def memoized_cut_rod(p, n):
    # p: price array, 1-indexed
    # n: rod length
    memo = [-1] * (n + 1)
    solution = [0] * (n + 1)

    best_value = memoized_cut_rod_aux(p, n, memo, solution)

    # reconstruct the cutting strategy
    cuts = []
    length = n
    while length > 0:
        cuts.append(solution[length])
        length -= solution[length]

    print("Top-down best revenue =", best_value)
    print("Cutting strategy =", cuts)
    return best_value, cuts


def memoized_cut_rod_aux(p, length, memo, solution):
    # base case
    if length == 0:
        return 0

    # already solved
    if memo[length] >= 0:
        return memo[length]

    # compute best
    best = -float("inf")
    for i in range(1, length + 1):
        current = p[i] + memoized_cut_rod_aux(p, length - i, memo, solution)
        if current > best:
            best = current
            solution[length] = i   # store the first cut

    memo[length] = best
    return best
n=9
#np.array([1,2,3,4,5,6,7,8,9,10],dtype=int)
p=[0,1,5,8,9,10,17,17,20,24,30]
print(memoized_cut_rod(p, n))

Top-down best revenue = 25
Cutting strategy = [3, 6]
(25, [3, 6])


In [ ]:
def rod_cut_bottom_up(p, n):
    dp = [0] * (n + 1)
    solution = [0] * (n + 1)

    for length in range(1, n + 1):
        best = -float('inf')
        for i in range(1, length + 1):
            if p[i] + dp[length - i] > best:
                best = p[i] + dp[length - i]
                solution[length] = i
        dp[length] = best

    # reconstruct
    cuts = []
    length = n
    while length > 0:
        cuts.append(solution[length])
        length -= solution[length]

    print("Bottom-up best revenue =", dp[n])
    print("Strategy =", cuts)
    return dp[n], cuts
n=9
#np.array([1,2,3,4,5,6,7,8,9,10],dtype=int)
p=[0,1,5,8,9,10,17,17,20,24,30]
print(rod_cut_bottom_up(p, n))

In [ ]:
def matrix_chain_order(p):
    n = len(p) - 1
    dp = [[0]* (n+1) for _ in range(n+1)]
    split = [[0]* (n+1) for _ in range(n+1)]

    for L in range(2, n+1):          # chain length
        for i in range(1, n-L+2):
            j = i + L - 1
            dp[i][j] = float('inf')
            for k in range(i, j):
                cost = dp[i][k] + dp[k+1][j] + p[i-1]*p[k]*p[j]
                if cost < dp[i][j]:
                    dp[i][j] = cost
                    split[i][j] = k

    print("Minimum multiplications =", dp[1][n])

    # reconstruct optimal parenthesization
    def print_opt(i, j):
        if i == j:
            return f"A{i}"
        k = split[i][j]
        return "(" + print_opt(i, k) + " x " + print_opt(k+1, j) + ")"

    print("Optimal parenthesization =", print_opt(1, n))

    return dp[1][n], print_opt(1, n)

In [ ]:
def knapsack(weights, values, W):
    n = len(weights)
    dp = [[0]*(W+1) for _ in range(n+1)]

    for i in range(1, n+1):
        for w in range(1, W+1):
            if weights[i-1] <= w:
                dp[i][w] = max(dp[i-1][w],
                               values[i-1] + dp[i-1][w-weights[i-1]])
            else:
                dp[i][w] = dp[i-1][w]

    # reconstruct chosen items
    chosen = []
    w = W
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i-1][w]:  # item i chosen
            chosen.append(i-1)
            w -= weights[i-1]

    chosen.reverse()

    print("Max value =", dp[n][W])
    print("Chosen items (0-indexed) =", chosen)

    return dp[n][W], chosen


In [ ]:
def lcs(X, Y):
    m, n = len(X), len(Y)

    # dp[i][j] = LCS length for X[:i] and Y[:j]
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Fill the dp table (bottom-up)
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if X[i - 1] == Y[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

    # Now reconstruct the LCS string by backtracking
    i, j = m, n
    lcs_chars = []

    while i > 0 and j > 0:
        # Characters match → this char is part of LCS
        if X[i - 1] == Y[j - 1]:
            lcs_chars.append(X[i - 1])
            i -= 1
            j -= 1
        # Move to the larger of the two neighbors
        elif dp[i - 1][j] >= dp[i][j - 1]:
            i -= 1
        else:
            j -= 1

    # Reverse because we collected from the end
    lcs_string = ''.join(reversed(lcs_chars))

    print("LCS length =", dp[m][n])
    print("LCS =", lcs_string)

    return dp[m][n], lcs_string


In [2]:
def lcs_length(X, Y):
    m = len(X)
    n = len(Y)

    # dp table, Python list is better than numpy for DP
    c = [[0] * (n + 1) for _ in range(m + 1)]

    # fill dp table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if X[i - 1] == Y[j - 1]:
                c[i][j] = c[i - 1][j - 1] + 1
            else:
                c[i][j] = max(c[i - 1][j], c[i][j - 1])

    # get actual LCS string by recursion
    lcs_str = print_lcs(c, X, Y, m, n)

    return c[m][n], lcs_str


def print_lcs(c, X, Y, i, j):
    if i == 0 or j == 0:
        return ""   # base case: empty string

    if X[i - 1] == Y[j - 1]:
        return print_lcs(c, X, Y, i - 1, j - 1) + X[i - 1]

    # choose direction based on dp values
    if c[i - 1][j] >= c[i][j - 1]:
        return print_lcs(c, X, Y, i - 1, j)
    else:
        return print_lcs(c, X, Y, i, j - 1)


In [3]:
X = "ABCBDAB"
Y = "BDCABA"

lcs_length(X, Y)


(4, 'BCBA')